Install relevant modules and packages

In [10]:
!apt-get install -y dcm2niix # '!' runs commands in the console
!apt-get install -y parallel
!apt-get install -y wget
!apt-get install -y grep
!apt-get install -y tcsh

!pip install nibabel

import os
import glob
import nibabel as nib

# install freesurfer (might take a few minutes)

# if you are having issues downloading freesurfer, it is likely due to the version selected here
!wget -O freesurfer.tar.gz https://freesurfer.net/pub/dist/freesurfer/7.4.1/freesurfer-linux-ubuntu22_amd64-7.4.1.tar.gz
!tar -xzf freesurfer.tar.gz

# set the relevant freesurfer directories
os.environ['FREESURFER_HOME'] = '/content/freesurfer'
os.environ['SUBJECTS_DIR'] = '/content/subjects'
os.environ['FS_LICENSE'] = '/content/license.txt'  # Add your license file path if you have one
os.environ['PATH'] += ':/content/freesurfer/bin'

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
dcm2niix is already the newest version (1.0.20211006-1build1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
parallel is already the newest version (20210822+ds-2).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
wget is already the newest version (1.21.2-2ubuntu1.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
grep is already the newest version (3.7-1build1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be

Get anonymized (T1 weighted) dicom files for analysis, and create temporary folders to store these and other files.

In [14]:
# clone the repository to get the raw dicoms
!git clone https://github.com/datalad/example-dicom-structural

# create folders for the NIfTI conversions and corresponding recons
!mkdir /content/nii_files/
!mkdir /content/freesurfer_output/

'''
Note: "T1 weighting" refers to a form of MRI image processing performed by technicians.
Online resources will nearly always provide processed MRI images,
though they may not be T1 weighted
''';

fatal: destination path 'example-dicom-structural' already exists and is not an empty directory.
mkdir: cannot create directory ‘/content/nii_files/’: File exists
mkdir: cannot create directory ‘/content/freesurfer_output/’: File exists


Convert the DICOMs to NIfTIs

In [15]:
# prepare paths
INPUT_PATH = '/content/example-dicom-structural/dicoms/'
OUTPUT_PATH = '/content/nii_files/'

# grep to ignore some warnings regarding the manufacturer (since we're using sample dicoms)
!dcm2niix -o '{OUTPUT_PATH}' '{INPUT_PATH}' | grep -v "Unknown manufacturer"

Chris Rorden's dcm2niiX version v1.0.20211006  (JP2:OpenJPEG) GCC11.2.0 x86-64 (64-bit Linux)
Found 384 DICOM file(s)
Convert 384 DICOM as /content/nii_files/dicoms_anat-T1w_20130717141500_401c (274x384x384x1)
Conversion required 0.394708 seconds (0.374460 for core code).


In [18]:
!file /content/freesurfer/python/bin/python3
!uname -m


/content/freesurfer/python/bin/python3: symbolic link to ../miniconda3_apple_dist/py3_8_13/bin/python3
x86_64


Reconstruct the NIfTI files.

Parallel allows you to recon multiple subjects simultaneously by recruiting n CPU cores (determined by --jobs n)

In [16]:
# prepare paths
INPUT_PATH = '/content/drive/MyDrive/freesurfer_output'
nii_paths = glob.glob('/content/drive/MyDrive/nii_files/*')
ALL_NIIS = ' '.join(nii_paths) # format that parallel wants

# Use parallel to execute recon-all on each NIfTI file
!parallel --jobs 2 'recon-all -i {} -sd {} -all' ::: $ALL_NIIS ::: "$INPUT_PATH"

ERROR: could not configure recon-all parameters
/content/freesurfer/bin/fspython: line 30: /content/freesurfer/python/bin/python3: cannot execute binary file: Exec format error
/content/freesurfer/bin/fspython: line 30: /content/freesurfer/python/bin/python3: Success
